In [1]:
# 导入 pysam 库，提供对 BAM 文件和其他生物信息学文件格式的操作功能
import pysam

# 导入 pandas 库，常用于数据处理和分析，特别是表格数据
import pandas as pd

# 导入 re 库，用于处理正则表达式，可以用于字符串搜索和匹配
import re

# 导入 numpy 库，提供支持大型多维数组和矩阵的计算功能
import numpy as np

# 从 pandas 库中导入 DataFrame 类，用于创建和操作数据帧
from pandas import DataFrame

# 导入 torch.nn.functional 模块，提供神经网络的各种函数，特别是激活函数、损失函数等
import torch.nn.functional as F

# 导入 polars 库，一个高性能的数据框架库，类似于 pandas，但速度更快
import polars

# 导入 gtfparse 库，用于解析 GTF 文件，通常用于基因注释
from gtfparse import read_gtf

# 从 scipy.stats 导入 pearsonr 函数，用于计算皮尔逊相关系数
from scipy.stats import pearsonr

# 从 torch.utils.data 导入 Dataset 和 DataLoader 类，用于创建数据集和数据加载器
from torch.utils.data import Dataset, DataLoader

# 导入 torch 库，这是一个深度学习框架
import torch

# 导入 torch.nn 模块，提供神经网络层、容器等的定义
import torch.nn as nn

# 从 scipy.stats 导入 pearsonr 函数，用于计算皮尔逊相关系数
# (这行重复了前面的导入，可以删除)
from scipy.stats import pearsonr


ModuleNotFoundError: No module named 'polars'

In [2]:
# 定义一个继承自 PyTorch 的 Dataset 类的自定义数据集 GenomicDataset
class GenomicDataset(Dataset):
    def __init__(self, gtf_path, fasta_path, tsv_path=None, gene_type=None, feature=None, output_type='onehot', include_tpm_fpkm=False, transform=None):
        """
        初始化函数，接受多种参数来设置数据集的属性。
        
        参数:
            gtf_path (str): GTF 文件的路径。
            fasta_path (str): FASTA 文件的路径。
            tsv_path (str, optional): 包含 TPM 和 FPKM 值的 TSV 文件路径。
            gene_type (str, optional): 用于过滤的特定基因类型，例如 'protein_coding'。
            feature (str, optional): 用于过滤的特定特征，例如 'exon'。
            output_type (str, optional): 所需的输出类型（'raw'，'onehot'，'all'）。
            include_tpm_fpkm (bool, optional): 如果为 True，在输出中包含 TPM 和 FPKM 值。
            transform (callable, optional): 可选的对样本进行处理的变换函数。
        """
        self.gtf_path = gtf_path
        self.fasta_path = fasta_path
        self.tsv_path = tsv_path
        self.gene_type = gene_type
        self.feature = feature
        self.output_type = output_type
        self.include_tpm_fpkm = include_tpm_fpkm
        self.transform = transform
        self.data = self.load_data()  # 加载并处理 GTF 文件数据
        self.tsv_data = self.load_tsv_data(tsv_path)  # 加载 TSV 文件数据
        self.fasta_file = pysam.FastaFile(self.fasta_path)  # 打开 FASTA 文件

    def load_data(self):
        """
        加载并处理 GTF 文件，根据提供的 gene_type 和 feature 进行过滤。
        """
        df = read_gtf(self.gtf_path)  # 读取 GTF 文件
        df = polars.DataFrame.to_pandas(df)  # 将 polars 数据帧转换为 pandas 数据帧

        # 如果指定了特征过滤器，则应用过滤器
        if self.feature:
            df = df[df["feature"] == self.feature]

        # 如果指定了基因类型过滤器，则应用过滤器
        if self.gene_type:
            df = df[df["gene_type"] == self.gene_type]
    
        return df  # 返回处理后的数据帧

    def load_tsv_data(self, tsv_path):
        """
        加载包含 TPM 和 FPKM 值的 TSV 文件。
        """
        tsv_data = pd.read_csv(tsv_path, sep='\t')  # 读取 TSV 文件
        return tsv_data  # 返回读取的数据

    def one_hot_encode(self, sequence):
        """
        将核苷酸序列转换为 one-hot 编码。
        """
        mapping = {
            'A': [1, 0, 0, 0, 0],
            'C': [0, 1, 0, 0, 0],
            'G': [0, 0, 1, 0, 0],
            'T': [0, 0, 0, 1, 0],
            'N': [0, 0, 0, 0, 1]
        }
        one_hot = np.array([mapping.get(nuc, [0, 0, 0, 0, 0]) for nuc in sequence], dtype=np.float32)  # 转换为 one-hot 编码
        return torch.tensor(one_hot)  # 转换为 PyTorch 张量

    def log_transform(self, value):
        """
        应用 log10(x + 1) 变换。
        """
        return np.log10(value + 1)  # 返回变换后的值

    def reverse_complement(self, sequence):
        """
        返回给定 DNA 序列的反向互补序列。
        """
        complement = str.maketrans('ACGTNacgtn', 'TGCANtgcan')  # 创建碱基互补关系表
        return sequence.translate(complement)[::-1]  # 返回反向互补序列

    def __len__(self):
        """
        返回数据集的长度。
        """
        return len(self.data)  # 返回数据帧的长度

    def __getitem__(self, idx):
        """
        根据索引返回一个数据样本。
        """
        record = self.data.iloc[idx]  # 获取指定索引的数据记录
        
        # 从 GTF 记录中提取信息
        chrom = record["seqname"]
        start = record["start"]
        end = record["end"]
        strand = record["strand"]
        gene_id = record["gene_id"]
        
        # 从 FASTA 文件中获取序列
        if strand == '-':
            sequence = self.fasta_file.fetch(chrom, end - 501, end + 499)
            sequence = self.reverse_complement(sequence)
        else:
            sequence = self.fasta_file.fetch(chrom, start - 499, start + 501)
        
        sample = {'sequence': None}  # 初始化输出字典

        # 确定输出类型
        if self.output_type == 'onehot':
            sample['sequence'] = self.one_hot_encode(sequence)
        elif self.output_type == 'all':
            sample['sequence'] = {'raw': sequence, 'onehot': self.one_hot_encode(sequence)}
        else:  # 默认输出 'raw'
            sample['sequence'] = sequence

        # 可选地包含 TPM 和 FPKM
        if self.include_tpm_fpkm:
            sample['tpm'] = None
            sample['fpkm'] = None
            if not self.tsv_data.empty and gene_id in self.tsv_data['gene_id'].values:
                row = self.tsv_data[self.tsv_data['gene_id'] == gene_id]
                tpm = row['TPM'].values[0] if not row.empty else None
                fpkm = row['FPKM'].values[0] if not row.empty else None

                # 应用 log10(x + 1) 变换
                sample['tpm'] = self.log_transform(tpm) if tpm is not None else None
                sample['fpkm'] = self.log_transform(fpkm) if fpkm is not None else None

        if self.transform:
            sample = self.transform(sample)  # 应用可选的变换
        
        return sample  # 返回样本
    
    def __del__(self):
        """
        析构函数，关闭 FASTA 文件。
        """
        self.fasta_file.close()


In [ ]:
if __name__ == "__main__":
    # 主程序入口，当脚本作为主程序运行时执行以下代码

    # 设置 GTF 文件路径（需要替换为实际路径）
    gtf_path = ''

    # 设置 FASTA 文件路径（需要替换为实际路径）
    fasta_path = ''

    # 设置 TSV 文件路径（需要替换为实际路径）
    tsv_path = ''

    # 自定义基因类型，过滤特定类型的基因（例如 'protein_coding'）
    gene_type = 'protein_coding'  # 自定义基因类型

    # 自定义特征类型，过滤特定特征（例如 'gene'）
    feature = 'gene'  # 自定义特征

    # 设置输出类型，可选值为 'raw', 'onehot', 'all'
    output_type = 'onehot'  # 设置输出类型

    # 指定是否包含 TPM 和 FPKM 值
    include_tpm_fpkm = True  # 包含 TPM 和 FPKM 值

    # 创建 GenomicDataset 数据集实例，传入各项参数
    dataset = GenomicDataset(gtf_path=gtf_path, fasta_path=fasta_path, tsv_path=tsv_path,
                             gene_type=gene_type, feature=feature, output_type=output_type,
                             include_tpm_fpkm=include_tpm_fpkm)

    # 创建 DataLoader 实例，用于批量加载数据
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True)  # 设置批量大小为4，并且随机打乱数据

    # 初始化空列表，用于存储不同批次的数据
    a = []  # 存储 transformed TPM values 的列表
    b = []  # 存储 transformed FPKM values 的列表
    c = []  # 存储 one-hot encoded sequences 的列表

    # 遍历 DataLoader 加载的数据批次
    for batch in dataloader:
        # 从当前批次中获取序列数据
        sequences = batch['sequence']

        # 从当前批次中获取 TPM 和 FPKM 数据，如果有的话
        tpm = batch.get('tpm')
        fpkm = batch.get('fpkm')

        # 将序列数据添加到列表 c 中
        c.append(sequences)  # 存储 one-hot 编码序列的张量

        # 如果包含 TPM 和 FPKM 值，将它们分别添加到列表 a 和 b 中
        if include_tpm_fpkm:
            a.append(tpm)    # 存储 transformed TPM 值的张量
            b.append(fpkm)   # 存储 transformed FPKM 值的张量
